<a href="https://colab.research.google.com/github/wilsonandres2602/LenguajesIUE/blob/main/Temperaturas_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import concurrent.futures
from estadisticas import calcular_estadisticas  # Invocacion libreria estadisticas/calcular_estadisticas

# URL de la API para obtener las temperaturas
API_URL = "https://weather.siel.com.co/city/{city}/temp/max"
TOKEN = "Bearer secret-token-1234"

# Función para obtener las temperaturas de 10 años
def obtener_temperaturas_10_anos(city):
    temperaturas_max = []
    for año in range(2013, 2023):  # Asumiendo que los datos van del 2013 al 2022
        try:
            response = requests.get(f"{API_URL.format(city=city)}?year={año}", headers={'Authorization': TOKEN})
            response.raise_for_status()  # Lanza un error si el código de estado no es 200
            temperaturas_max.append(response.json()['temperature'])  # Suponiendo que la API devuelve 'temperature'
        except requests.exceptions.RequestException as e:
            print(f"Error al obtener datos de {city} para el año {año}: {e}")
            temperaturas_max.append(None)  # Si falla, agregamos None

    return temperaturas_max

# Función para obtener las temperaturas de varias ciudades de forma concurrente
def obtener_temperaturas_concurrentes(cities):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        temperaturas = list(executor.map(obtener_temperaturas_10_anos, cities))
    return temperaturas

# Función principal para calcular las estadísticas de las temperaturas
def calcular_estadisticas_temperaturas(cities):
    temperaturas = obtener_temperaturas_concurrentes(cities)
    # Filtramos las temperaturas válidas (no None)
    temperaturas_validas = [temp for temp in temperaturas if None not in temp]  # Removemos ciudades con None en las temperaturas

    if len(temperaturas_validas) > 0:
        # Flatten the list of temperatures (flatten to get all temperatures in one list)
        all_temperatures = [temp for sublist in temperaturas_validas for temp in sublist]
        estadisticas = calcular_estadisticas(all_temperatures)  # Utilizamos la librería de estadísticas
        return estadisticas
    else:
        return "No se obtuvieron temperaturas válidas."

# Lista de ciudades para obtener las temperaturas
cities = ["paris", "new_york", "madrid", "london", "tokyo", "sydney", "beijing", "moscow", "berlin", "dubai"]

# Calcular y mostrar las estadísticas
estadisticas = calcular_estadisticas_temperaturas(cities)

# Imprimir los resultados
print("\nEstadísticas de temperaturas obtenidas:")
print(estadisticas)
